In [ ]:
"""Compute mean and standard deviation of the evaluation metrics and training times
   of each trained model on the C-MAPSS dataset."""

import os
from datetime import datetime, timedelta

import numpy as np
import math

DATASET = "CMAPSS/FD001"
PATH = DATASET + "/min-max"

MAX_RUL = 10000 # 125

logs = [os.path.join(root, f)
        for root, dirs, files in os.walk(PATH)
        for f in files if f[-3:] == "txt"]

MODELS = ["frequentist_dense3",
          "bayesian_dense3",
          "frequentist_conv2_pool2",
          "bayesian_conv2_pool2",
          "frequentist_conv5_dense1",
          "bayesian_conv5_dense1_1.00",
          "bayesian_conv5_dense1_0.50",
          "bayesian_conv5_dense1_0.25",
          "bayesian_conv5_dense1_0.125"] \
          \
          if DATASET in ["CMAPSS/FD001", "CMAPSS/FD002", "CMAPSS/FD004"] else \
          \
         ["frequentist_dense3",
          "bayesian_dense3_1.00",
          "bayesian_dense3_0.50",
          "bayesian_dense3_0.25",
          "bayesian_dense3_0.125",
          "frequentist_conv2_pool2",
          "bayesian_conv2_pool2",
          "frequentist_conv5_dense1",
          "bayesian_conv5_dense1"]
    
print(DATASET)
for MODEL in MODELS:
    rmses = []
    maes = []
    scores = []
    epistemics = []
    training_times = []
    for log in logs:
        if MODEL in log:
            # evaluation log
            if "log_evaluate.txt" in log:
                with open(log) as f:
                    content = f.read()
                sse = 0
                sae = 0
                score = 0
                epistemic = 0
                i = content.find("ground truth | pred +/- std:") + 29
                count = 0
                while content[i] != "e":
                    label_end = i + content[i:].find(" | ")
                    pred_end = i + content[i:].find(" +/- ")
                    std_end = i + content[i:].find("\n")
                    label = float(content[i : label_end])
                    pred = float(content[label_end + 3 : pred_end])
                    # rectify label
                    if label > MAX_RUL:
                        label = MAX_RUL
                    delta = pred - label
                    sse += delta ** 2
                    sae += np.abs(delta)
                    score += np.exp(-delta / 13) - 1 if delta < 0 else np.exp(delta / 10) - 1
                    epistemic += float(content[pred_end + 5 : std_end]) ** 2
                    i += content[i:].find("\n") + 1
                    count += 1
                rmses.append(np.sqrt(sse / count))
                maes.append(sae / count)
                scores.append(score)
                epistemics.append(epistemic / count)

            # training log
            elif "log_train.txt" in log:
                with open(log) as f:
                    content = f.readlines()[-3]
                i = content.find("elapsed time: ") + 14
                t = datetime.strptime(content[i:].split("\n")[0], "%H:%M:%S.%f")
                delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond).total_seconds()
                training_times.append(delta)

    print("\n" + MODEL)
    print("rmse: %.2f +/- %.2f" % (np.mean(rmses), np.std(rmses)))
    print("mae: %.2f +/- %.2f" % (np.mean(maes), np.std(maes)))
    print("score: %d +/- %d" % (np.mean(scores), np.std(scores)))
    print("epistemic: %.2f +/- %.2f" % (np.mean(epistemics), np.std(epistemics)))
    print("training time: %.2f +/- %.2f" % (np.mean(training_times), np.std(training_times)))